In [2]:
import pandas as pd
from IPython.display import display

# Import csv
customers = pd.read_csv("../data/raw/olist_customers_dataset.csv")
geolocation = pd.read_csv("../data/raw/olist_geolocation_dataset.csv")
order_items = pd.read_csv("../data/raw/olist_order_items_dataset.csv")
order_payments = pd.read_csv("../data/raw/olist_order_payments_dataset.csv")
order_reviews = pd.read_csv("../data/raw/olist_order_reviews_dataset.csv")
orders = pd.read_csv("../data/raw/olist_orders_dataset.csv")
products = pd.read_csv("../data/raw/olist_products_dataset.csv")
sellers = pd.read_csv("../data/raw/olist_sellers_dataset.csv")
product_category_translation = pd.read_csv("../data/raw/product_category_name_translation.csv")




In [3]:
display(orders.head(3))

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


In [4]:
display(orders.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


None

In [5]:
print('Alterando tipos de coluna para data')
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])
display(orders.info())


Alterando tipos de coluna para data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


None

In [6]:
#Identificando quantos valores nulos tem em cada coluna
display(orders.isnull().sum())
#primeiro vamos limpar a tabela orders removendo valores nulos, duplicados

print('Com base nos dados obtemos que 160 produtos ainda não foram aprovados, então iremos excluir esses produtos da análise exploratória')

orders_filtred = orders.dropna(subset=['order_approved_at'])
display(orders_filtred.isnull().sum())

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Com base nos dados obtemos que 160 produtos ainda não foram aprovados, então iremos excluir esses produtos da análise exploratória


order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                   0
order_delivered_carrier_date     1637
order_delivered_customer_date    2819
order_estimated_delivery_date       0
dtype: int64

## 📦 Análise de Performance Logística

## 🚚 Tempo médio entre aprovação e entrega

In [7]:
# Análise de vendas:
#Tempo médio entre a compra e a entrega (order_purchase_timestamp → order_delivered_customer_date)

average_time = (orders_filtred['order_delivered_customer_date'] - orders_filtred['order_approved_at']).mean().days
print(f'O tempo médio de entrega após a aprovação da compra é de {average_time} dias')

O tempo médio de entrega após a aprovação da compra é de 12 dias


In [8]:
display(orders_filtred.head(1))


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18


In [9]:
#Porcentagem de entregas atrasadas por categoria de produto, estado e vendedor

total_delays = (orders_filtred['order_delivered_customer_date'] > orders_filtred['order_estimated_delivery_date']).sum()

print(f'O total de pedidos é: {len(orders_filtred)}')
print(f'Total de pedidos entregues com atraso: {total_delays}')

#porcentagem de atrasados sobre total de pedidos:
percent = round((total_delays / len(orders_filtred)) * 100, 2)
print(f'Porcentagem de atrasados sobre total de pedidos: {percent}%')


#Porcentagem de entregas atrasadas por categoria de produto

#tabela products

O total de pedidos é: 99281
Total de pedidos entregues com atraso: 7827
Porcentagem de atrasados sobre total de pedidos: 7.88%


In [10]:
display(products.head(1))
display(orders_filtred.head(1))
display(order_items.head(1))



,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29


In [11]:
# Unindo order_items com products para ter a categoria de produto
merged_items = pd.merge(order_items, products, on='product_id', how='inner')

#Criando uma copy de orders_filtred
orders_filtred2 = orders_filtred.copy()

# Unindo com orders_filtred para ter as informações dos pedidos
merged_data = pd.merge(orders_filtred2, merged_items, on='order_id', how='inner')

# Filtrando os pedidos atrasados
delayed_orders = merged_data[merged_data['order_delivered_customer_date'] > merged_data['order_estimated_delivery_date']]

#Contando apenas pedidos unicos com o mesmo order_id
display(delayed_orders['order_id'].nunique())


7827

In [ ]:
# 1. Total de pedidos atrasados por categoria
delayed_counts = delayed_orders.groupby('product_category_name')['order_id'].nunique().reset_index()
delayed_counts.columns = ['product_category_name', 'delayed_orders']

# 2. Total de pedidos por categoria (com base em pedidos aprovados)
total_counts = merged_data.groupby('product_category_name')['order_id'].nunique().reset_index()
total_counts.columns = ['product_category_name', 'total_orders']

# 3. Unir os dois dataframes pelo nome da categoria
delay_stats = delayed_counts.merge(total_counts, on='product_category_name')



delay_stats['delay_percentage'] = delay_stats['delayed_orders'] / delay_stats['total_orders']

# Arredonda para quatro casas decimais
delay_stats['delay_percentage'] = delay_stats['delay_percentage'].round(4)


delay_stats.to_csv('delayed_orders_by_category.csv', index=False, float_format='%.4f', decimal=',')






print(delay_stats.to_string(index=False))


                         product_category_name  delayed_orders  total_orders  delay_percentage
                     agro_industria_e_comercio               9           182            0.0495
                                     alimentos              44           450            0.0978
                             alimentos_bebidas              14           227            0.0617
                                         artes              15           202            0.0743
                            artes_e_artesanato               2            23            0.0870
                             artigos_de_festas               1            39            0.0256
                              artigos_de_natal              12           128            0.0938
                                         audio              45           350            0.1286
                                    automotivo             328          3897            0.0842
                                         bebes    